In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
from retinanet import paps_train

# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.anchors import *
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.hourglass import hg1, hg2, hg8
from retinanet.parallel import DataParallelModel, DataParallelCriterion

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim

# python train_paps.py --start_epoch 0 --end_epoch 120 --batch_size 24 \
#                     --saved_dir $OUT_MODEL_DIR --gpu_num 0 --num_workers 12 \
#                     --target_threshold 7 --topk 20 --filter_option 1  | 2>&1 | tee $log


In [2]:
# device = torch.device('cpu')
# device = torch.device('cuda')
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
model = hg2(device, pretrained=True, progress=False, num_classes=2)
model.to(device)

num_classes 2
num_anchors per feature map 1


HourglassNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
  (layer2): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=Tr

In [3]:
# model

In [4]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 1
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
    pin_memory=True,
    prefetch_factor=1,
    collate_fn=collate_fn
    
)

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!


In [5]:
criterion = PapsLoss(device, target_threshold=8, topk=10, filter_option=6)
criterion = criterion.to(device)
model.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(model.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [6]:
saved_dir = '../trained_models/HourGlass/loss_image_filter' + str(0) + '/'
s_epoch= 0
e_epoch= 1

In [7]:
if os.path.isdir(saved_dir) == False :
    print('folder is made')
    os.makedirs(saved_dir)

In [8]:
if os.path.isfile(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt') :
    print('pretrainind file loading')
    state = torch.load(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt')
    epoch = state['epoch']
    model.load_state_dict(state['state_dict'], strict=False)
    optimizer.load_state_dict(state['optimizer'])
    last_loss = state['loss']
else :
    last_loss = 0.6

In [9]:
# images = images.cpu()
# images = [torchvision.transforms.ToPILImage()(x) for x in images]
# images = [torchvision.transforms.Grayscale()(x) for x in images]
# images = [torchvision.transforms.ToTensor()(x) for x in images]
# images = torch.stack(images).to(device)

In [ ]:
if s_epoch == 0:
    Iou_low = [0.2]
    beta_list = [1.0]
    for i in range(1, e_epoch+1) :
        Iou_low.append(Iou_low[0] + (0.5-0.2)*i/160)
        beta_list.append(beta_list[0] - (1.0-0.5)*i/160)   

loss_per_epoch = last_loss
cls_loss = []
box_loss = []
for epoch in range(s_epoch, e_epoch) :
    total_loss = 0
    tk0 = tqdm(train_data_loader, total=len(train_data_loader), leave=False)
    EPOCH_LEARING_RATE = optimizer.param_groups[0]["lr"]
    foreground_class = []
    background_class = []

    for step, data in enumerate(tk0) :
        if step > len(train_data_loader)/2 and epoch%10 > 0 :
#             if step > len(train_data_loader)/4 :
            break 
        if s_epoch == 0 :
            iou_thres = np.random.uniform(low=Iou_low[epoch], high=0.5, size=None)
            beta = np.random.uniform(low=beta_list[epoch]-0.1, high=beta_list[epoch], size=None)
        else :
            iou_thres = 0.5
            beta = 0.5
        images, box, label, targets = data
        batch_size = len(images)
#         pimages = images.cpu()
#         print(pimages.shape)

        c, h, w = images[0].shape
        pimages = PyramidImages()(images).to(device)
        images = torch.cat(images).view(-1, c, h, w).to(device)
        targets = [ t.to(device) for t in targets]

        outputs = model([images, targets])
        
        
#         print(pimages.shape)
        
        classification, regression, anchors, _ = (outputs)
        
        classification_loss, regression_loss, num_det, back_c, fore_c = criterion(classification, regression, 
                                                         anchors, targets, iou_thres,
                                                         beta, pimages=pimages)
#         print(fore_c)
        foreground_class.extend(fore_c)
        background_class.extend(back_c)

        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()
        loss = classification_loss + regression_loss 
        total_loss += loss.item()
        cls_loss.append(classification_loss.item())
        box_loss.append(regression_loss.item())

        if step % 1 == 0:
            tk0.set_postfix(lr=optimizer.param_groups[0]["lr"], batch_loss=loss.item(), cls_loss=classification_loss.item(), 
                            reg_loss=regression_loss.item(), avg_loss=total_loss/(step+1), num_det=num_det)        

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()   

    print('{}th epochs loss {} lr {} '.format(epoch, total_loss/(step+1), EPOCH_LEARING_RATE ))
    if loss_per_epoch > total_loss/(step+1):
        print('best model is saved')
        torch.save(model.state_dict(), saved_dir + 'best_model.pt')
        loss_per_epoch = total_loss/(step+1)

    scheduler.step()

 22%|██▏       | 3206/14874 [07:59<30:24,  6.40it/s, avg_loss=0.916, batch_loss=1, cls_loss=0.988, lr=1e-8, num_det=tensor(11, device='cuda:0'), reg_loss=0.0118]       

In [ ]:
len(background_class)

In [ ]:
import time
# len(background_class)
def my_def ( i) :
    i = i.cpu()
    i = np.array(i)
    return i

stime = time.time()
bg_cls = list(map(my_def, background_class))
print('time', time.time() - stime)
len(bg_cls)

In [ ]:
fore_cls = list(map(my_def, foreground_class))
len(fore_cls)

In [ ]:
sum(background_class)/len(background_class)
# background_class

In [ ]:
# (foreground_class)
# for_list = []
# for i in foreground_class :
#     if (i) != torch.tensor(0) :
#         for_list.append(i.numpy())
# # sum(foreground_class)/len(foreground_class)
# sum(for_list)/len(for_list)
# # for_list

In [ ]:
fore_cls[0:10]

In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt

# data = np.random.normal(0, 1, 1000) 
# data = np.array(background_class)
data = np.array(fore_cls)
# fixed bin size
bins = np.arange(0, 1, 1/20) # fixed bin size

plt.xlim([0, 1])

plt.hist(data, bins=bins, alpha=0.5)
plt.title('Random Gaussian data (fixed bin size)')
plt.xlabel('variable X (bin size = 5)')
plt.ylabel('count')

plt.show()

In [ ]:
(bg_cls[0:10])

In [ ]:
# data = np.random.normal(0, 1, 1000) 
data = np.array(bg_cls)
# data = np.array(for_list)
# fixed bin size
bins = np.arange(0, 1, 1/50) # fixed bin size

plt.xlim([0, 1])

stime = time.time()
plt.hist(data, bins=bins, alpha=0.5)
plt.title('Random Gaussian data (fixed bin size)')
plt.xlabel('variable X (bin size = 5)')
plt.ylabel('count')
plt.show()
print('time', time.time() - stime)

In [ ]:
a = torch.randn(2, 2000)
a.shape
a[1,:].shape

In [ ]:
# pool of square window of size=3, stride=2
# m = nn.MaxPool2d((8,8), stride=8)
# pool of non-square window
m = nn.AvgPool2d((32, 32), stride=(8, 8), padding=12)
input = torch.randn(24, 3, 320, 320)
output = m(input)
output.shape
output.mean(dim=1)

In [ ]:
class Anchors(nn.Module):
    def __init__(self, pyramid_levels=None, strides=None, sizes=None, ratios=None, scales=None):
        super(Anchors, self).__init__()

        if pyramid_levels is None:
#             self.pyramid_levels = [3, 4, 5, 6, 7]
            self.pyramid_levels = [3, 4]
        if strides is None:
            self.strides = [2 ** x for x in self.pyramid_levels]
        if sizes is None:
            self.sizes = [2 ** (x + 2) for x in self.pyramid_levels]
        if ratios is None:
            self.ratios = np.array([1, 2])
#             self.ratios = np.array([0.5, 0.75, 1, 1.25, 1.5])
        if scales is None:
            self.scales = np.array([2 ** 0, 2 ** 1 ])
#             self.scales = np.array([2 ** (-1.0 / 3.0), 2 ** (-3.0 / 3.0), 2 ** 0, 2 ** (1.0 / 3.0) ])
#         num anchors per feature map
        self.num_anchors = len(self.ratios) * len(self.scales)

    def forward(self, image):
        
        image_shape = image.shape[2:]
        image_shape = np.array(image_shape)
        image_shapes = [(image_shape + 2 ** x - 1) // (2 ** x) for x in self.pyramid_levels]

        # compute anchors over all pyramid levels
        all_anchors = np.zeros((0, 4)).astype(np.float32)

        for idx, p in enumerate(self.pyramid_levels):
            anchors         = generate_anchors(base_size=self.sizes[idx], ratios=self.ratios, scales=self.scales)
            shifted_anchors = shift(image_shapes[idx], self.strides[idx], anchors)
            all_anchors     = np.append(all_anchors, shifted_anchors, axis=0)

        all_anchors = np.expand_dims(all_anchors, axis=0)

#         if torch.cuda.is_available():
#             return torch.from_numpy(all_anchors.astype(np.float32)).cuda()
#         else:
#             return torch.from_numpy(all_anchors.astype(np.float32))
        return torch.from_numpy(all_anchors.astype(np.float32))

In [ ]:
anchors = Anchors()(torch.randn(24, 3, 20, 20))
print(anchors.shape)
anchors

In [ ]:
paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

In [ ]:
# state = {
#     'epoch': 0,
#     'state_dict': model.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : 0.6
# }
# torch.save(state, saved_dir + 'model.pt')

In [ ]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 


In [ ]:
criterion = PapsLoss(device, target_threshold=9, topk=10, filter_option=1)
criterion = criterion.to(device)
s_epoch= 60
e_epoch= 120


paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

In [ ]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 
